### In this notebook, the json data input is loaded and reviewed and new structured data is formed. Also, the new data is explored for quality issues.

In [ ]:
#Import all libraries
import pandas as pd
import json
import numpy as np
import pandasql as ps

## Users data

In [ ]:
#load users json and display first few rows of users dataframe
users_list= []
for line in open('users.json','r'):
    users_list.append(json.loads(line))

users_df=pd.json_normalize(users_list)
users_df.head()

In [ ]:
#check the datatypes 
users_df.dtypes

### The lastLogin.$date column is changed to float during json_normalize because the columns contains NaN.

In [ ]:
#clean data that was changed
users_df['lastLogin.$date'] = pd.to_datetime(users_df['lastLogin.$date'],unit = 'ms')

In [ ]:
#convert epcoh time to datetime
users_df['createdDate.$date'] = pd.to_datetime(users_df['createdDate.$date'],unit='ms')

In [ ]:
#columns names contains special characters , replacing them 
users_df=users_df.rename(columns={"_id.$oid": "user_id","createdDate.$date":"createdDate.date","lastLogin.$date":"lastLogin.date"})
users_df.head()

In [ ]:
#check if rows are duplicated
users_df.duplicated()

In [ ]:
users_df.iloc[1:3,:]

In [ ]:
#drop duplicates
users=users_df.drop_duplicates(keep="first", inplace=False).reset_index(drop=True)
users.head()

In [ ]:
#save the new users data
users.to_csv("users.csv")

## Brands data

In [ ]:
#load data and display first few rows of the dataframe 
brands_list= []
for line in open('brands.json','r'):
    brands_list.append(json.loads(line))
brands_df=pd.json_normalize(brands_list)
brands_df.head()

In [ ]:
#columns names contains special characters , replacing them 
brands=brands_df.rename(columns={"_id.$oid": "brand_uuid","cpg.$ref":"cpg.ref","cpg.$id.$oid":"cpg.id.oid"})
brands.head()

In [ ]:
#check for duplicates
brands.duplicated().value_counts()

In [ ]:
#save new dataframe
brands.to_csv("brands.csv")

## Receipts data

In [ ]:
#load data and display first few rows of the dataframe
receipts_list= []
for line in open('receipts.json','r'):
    receipts_list.append(json.loads(line))
receipts_df=pd.json_normalize(receipts_list)
receipts_df.head()

In [ ]:
#changing datatypes for columns
receipts_df['totalSpent'] = receipts_df['totalSpent'].astype('float64')
receipts_df['purchasedItemCount'] = receipts_df['purchasedItemCount'].astype('Int64')
receipts_df['pointsEarned'] = receipts_df['pointsEarned'].astype('float64')

In [ ]:
#columns names contains special characters , replacing them 
receipts_df=receipts_df.rename(columns={"_id.$oid": "receipt_uuid","dateScanned.$date":"dateScanned",
                                       "createDate.$date":"createDate","finishedDate.$date":"finishedDate",
                                      "modifyDate.$date":"modifyDate","pointsAwardedDate.$date":"pointsAwardedDate",
                                      "purchaseDate.$date":"purchaseDate" })
receipts_df.head()

In [ ]:
#The date columns are changed to float during json_normalize because the columns contains NaN.
receipts_df.iloc[:,9:]=receipts_df.iloc[:,9:].apply(pd.to_datetime,unit='ms', errors='coerce')

In [ ]:
receipts_df.head()

## extracting Receiptitemlist from receipts 

In [ ]:
#check if there exist any null values in the two columns
ReceiptItemList=receipts_df[["rewardsReceiptItemList","receipt_uuid"]]
ReceiptItemList.isnull().values.any()


In [ ]:
receipts= receipts_df.drop(columns=['rewardsReceiptItemList'])
receipts.head()

In [ ]:
#check for duplicates
receipts.duplicated().value_counts()

In [ ]:
#check how many null values are present
ReceiptItemList["rewardsReceiptItemList"].isnull().sum()

In [ ]:
#replace Nan in receipt item list to empty list([])
ReceiptItemList["rewardsReceiptItemList"]=[[] if x is np.NaN else x for x in ReceiptItemList["rewardsReceiptItemList"]]
ReceiptItemList["rewardsReceiptItemList"].isnull().sum()

In [ ]:
#extracting and flattening nested json
ReceiptItemList_df = pd.concat({i: pd.json_normalize(x) for i, x in ReceiptItemList.pop('rewardsReceiptItemList').items()
                                   }).reset_index(level=1, drop=True).join(ReceiptItemList).reset_index()

In [ ]:
ReceiptItemList_df['quantityPurchased'] = ReceiptItemList_df['quantityPurchased'].astype('Int64')

In [ ]:
ReceiptItemList_df.head()

In [ ]:
ReceiptItemList_df=ReceiptItemList_df.rename(columns={"index":"receiptitem_id"})

In [ ]:
ReceiptItemList_df.head()

In [ ]:
ReceiptItemList_df.to_csv("receiptsitemlist.csv")

## We can observe lot of Nulls in the column fields , we can explore them in detail in the explore part of notebook

In [ ]:
ps.sqldf("select * from users")
ps.sqldf("select * from brands")
ps.sqldf("select * from receipts")
ps.sqldf("select * from ReceiptItemList_df");

# 1.What are the top 5 brands by receipts scanned for most recent month?

In [ ]:
query="""select distinct * from (
    select *, dense_rank() over(order by brand_cnt desc) as rnk from (select b.name, count(*) over(partition by b.name) as brand_cnt 
from receipts r 
join ReceiptItemList_df r2 on (r2.receipt_uuid = r.receipt_uuid)
join users u on (r.userid = u.user_id)
join brands b on (b.barcode = r2.barcode)
where datescanned BETWEEN datetime('now', '-30 days') AND datetime('now', 'localtime')
order by 2 desc
) t 
) t2 
where rnk <= 5
"""
ps.sqldf(query)

## Lets explore if the data has recent months data

In [ ]:
receipts[receipts["dateScanned"].dt.month=="5"]

In [ ]:
max(receipts["dateScanned"])

#### So there is no recent data . The data is till march 2021

# 2. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [ ]:
query2="""
select distinct * 
from 
(select *, dense_rank() over(order by brand_cnt desc) as rnk 
from 
(select b.name, count(*) over(partition by b.name) as brand_cnt 
from receipts r 
join ReceiptItemList_df r2 on (r2.receipt_uuid = r.receipt_uuid)
join users u on (r.userid = u.user_id)
join brands b on (b.barcode = r2.barcode)
where datescanned BETWEEN datetime('now', '-60 days') AND datetime('now', '-30 days')
order by 2 desc
) t 
) t2 
where rnk <= 5
"""
ps.sqldf(query2)

### As observed , the data is only till March 2021. So the above query results are empty 

# 3. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [ ]:
query3="""
select rewardsreceiptstatus, avg(totalspent) 
from receipts r 
where rewardsreceiptstatus in ('FINISHED', 'REJECTED')
group by 1 
order by 2 desc
"""
ps.sqldf(query3)

## Accepted is greater

# 4. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [ ]:
query4="""
select r2.rewardsreceiptstatus, sum(quantitypurchased) 
from ReceiptItemList_df r 
join receipts r2 on (r2.receipt_uuid = r.receipt_uuid)
where rewardsreceiptstatus in ('FINISHED', 'REJECTED')
group by 1 
order by 2 desc
"""
ps.sqldf(query4)

## Accepted is greater

# 5. Which brand has the most spend among users who were created within the past 6 months?

In [ ]:
query5="""select b."name", sum(quantitypurchased) 
from receipts r 
join ReceiptItemList_df r2 on (r2.receipt_uuid = r.receipt_uuid)
join users u on (r.userid = u.user_id)
join brands b on (b.barcode = r2.barcode)
where u."createdDate.date" BETWEEN datetime('now', '-6 months') AND datetime('now', 'localtime')
group by 1
order by 2 desc
"""
ps.sqldf(query5)

# 6. Which brand has the most transactions among users who were created within the past 6 months?

In [ ]:
query6="""select b."name", count(*)
from receipts r 
join ReceiptItemList_df r2 on (r2.receipt_uuid = r.receipt_uuid)
join users u on (r.userid = u.user_id)
join brands b on (b.barcode = r2.barcode)
where u."createdDate.date" BETWEEN datetime('now', '-6 months') AND datetime('now', 'localtime')
group by 1
order by 2 desc
"""
ps.sqldf(query6)

# Exploring data for quality issues

### So far we observed, 
#### 1. The column names have special characters and need to renamed to be readable.
#### 2. user_id is not unique in users dataframe and has lot of duplicates
#### 3. There are lot of Null values for rows for different columns, we need to treat Null values before we apply any modeling techniques
#### or use data for analysis. The data can be replaced by mean ,median or mode or remove the rows that do 
#### not add value for analysis or modeling.

In [ ]:
#lets compare userid in receipts and users dataframe
print(" Number of users", users['user_id'].nunique())
print(" Number of users in receipts df", receipts['userId'].nunique())

#### Based on this observation, receipts has some users that are not in users dataframe

In [ ]:
print("Number of users present in receipts table that are not in users table:",len(set(receipts["userId"].unique())-set(users["user_id"].unique())))

In [ ]:
#users that are not in users table but present in receipts
set(receipts["userId"].unique())-set(users["user_id"].unique())

### we have used barcode as Foreign key to join brands and receiptitemlist

In [ ]:
len(ReceiptItemList_df)

In [ ]:
ReceiptItemList_df.iloc[1:7,:]

###  We can observe there are receipts with barcode not found and barcode 1234 for the ones that dont have description

In [ ]:
ReceiptItemList_df["barcode"].isnull().sum()

### we can notice lot of records having null values for barcode in ReceiptItemList_df, which is important field as each receipt is associated with a unique barcode to identify items and there brands

In [ ]:
brands["name"].value_counts()

In [ ]:
brands[brands["name"]=="Diabetic Living Magazine"]

### Same brand name is associated with different barcodes.


## Overall, the data is lacking consistency 